In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
path_to_file = tf.keras.utils.get_file('/content/Bharathiyar_Poems_Excerpts.txt',"https://www.goodreads.com/work/quotes/14689534")

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 13126 characters


In [6]:
print(text[:250])

“நிமிர்ந்த நன்னடை நேர்கொண்ட பார்வையும்,
நிலத்தில் யார்க்கும் அஞ்சாத நெறிகளும்,
திமிர்ந்த ஞானச் செருக்கும் இருப்பதால்
செம்மை மாதர் திறம்புவ தில்லையாம்;
அமிழ்ந்து பேரிரு ளாமறி யாமையில்
அவல மெய்திக் கலையின் றி வாழ்வதை
உமிழ்ந்து தள்ளுதல் பெண்ணற மாகுமாம்



In [7]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

51 unique characters


In [8]:
print(vocab)

['\n', ' ', '!', ',', '-', '.', ';', '?', 'அ', 'ஆ', 'இ', 'உ', 'எ', 'ஏ', 'ஒ', 'ஓ', 'க', 'ங', 'ச', 'ஞ', 'ட', 'ண', 'த', 'ந', 'ன', 'ப', 'ம', 'ய', 'ர', 'ற', 'ல', 'ள', 'ழ', 'வ', 'ஸ', 'ஹ', 'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', '்', '‘', '’', '“', '”']


In [10]:
example_texts = ['நிமிர்ந்த', 'வாழ்வதை']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'\xe0\xae\xa8', b'\xe0\xae\xbf', b'\xe0\xae\xae', b'\xe0\xae\xbf',
  b'\xe0\xae\xb0', b'\xe0\xaf\x8d', b'\xe0\xae\xa8', b'\xe0\xaf\x8d',
  b'\xe0\xae\xa4']                                                   ,
 [b'\xe0\xae\xb5', b'\xe0\xae\xbe', b'\xe0\xae\xb4', b'\xe0\xaf\x8d',
  b'\xe0\xae\xb5', b'\xe0\xae\xa4', b'\xe0\xaf\x88']                 ]>

In [11]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [13]:
#it convers from tokens to character IDs
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[24, 38, 27, 38, 29, 47, 24, 47, 23], [34, 37, 33, 47, 34, 23, 44]]>

In [15]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'\xe0\xae\xa8', b'\xe0\xae\xbf', b'\xe0\xae\xae', b'\xe0\xae\xbf',
  b'\xe0\xae\xb0', b'\xe0\xaf\x8d', b'\xe0\xae\xa8', b'\xe0\xaf\x8d',
  b'\xe0\xae\xa4']                                                   ,
 [b'\xe0\xae\xb5', b'\xe0\xae\xbe', b'\xe0\xae\xb4', b'\xe0\xaf\x8d',
  b'\xe0\xae\xb5', b'\xe0\xae\xa4', b'\xe0\xaf\x88']                 ]>

In [16]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [17]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(13126,), dtype=int64, numpy=array([50, 24, 38, ..., 51,  1,  1])>

In [18]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [19]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

“
ந
ி
ம
ி
ர
்
ந
்
த


In [20]:
seq_length = 100

In [32]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))


tf.Tensor(
[b'\xe2\x80\x9c' b'\xe0\xae\xa8' b'\xe0\xae\xbf' b'\xe0\xae\xae'
 b'\xe0\xae\xbf' b'\xe0\xae\xb0' b'\xe0\xaf\x8d' b'\xe0\xae\xa8'
 b'\xe0\xaf\x8d' b'\xe0\xae\xa4' b' ' b'\xe0\xae\xa8' b'\xe0\xae\xa9'
 b'\xe0\xaf\x8d' b'\xe0\xae\xa9' b'\xe0\xae\x9f' b'\xe0\xaf\x88' b' '
 b'\xe0\xae\xa8' b'\xe0\xaf\x87' b'\xe0\xae\xb0' b'\xe0\xaf\x8d'
 b'\xe0\xae\x95' b'\xe0\xaf\x8a' b'\xe0\xae\xa3' b'\xe0\xaf\x8d'
 b'\xe0\xae\x9f' b' ' b'\xe0\xae\xaa' b'\xe0\xae\xbe' b'\xe0\xae\xb0'
 b'\xe0\xaf\x8d' b'\xe0\xae\xb5' b'\xe0\xaf\x88' b'\xe0\xae\xaf'
 b'\xe0\xaf\x81' b'\xe0\xae\xae' b'\xe0\xaf\x8d' b',' b'\n'
 b'\xe0\xae\xa8' b'\xe0\xae\xbf' b'\xe0\xae\xb2' b'\xe0\xae\xa4'
 b'\xe0\xaf\x8d' b'\xe0\xae\xa4' b'\xe0\xae\xbf' b'\xe0\xae\xb2'
 b'\xe0\xaf\x8d' b' ' b'\xe0\xae\xaf' b'\xe0\xae\xbe' b'\xe0\xae\xb0'
 b'\xe0\xaf\x8d' b'\xe0\xae\x95' b'\xe0\xaf\x8d' b'\xe0\xae\x95'
 b'\xe0\xaf\x81' b'\xe0\xae\xae' b'\xe0\xaf\x8d' b' ' b'\xe0\xae\x85'
 b'\xe0\xae\x9e' b'\xe0\xaf\x8d' b'\xe0\xae\x9a' b'\xe0\xae

In [33]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy().decode('utf-8'))

“நிமிர்ந்த நன்னடை நேர்கொண்ட பார்வையும்,
நிலத்தில் யார்க்கும் அஞ்சாத நெறிகளும்,
திமிர்ந்த ஞானச் செருக்
கும் இருப்பதால்
செம்மை மாதர் திறம்புவ தில்லையாம்;
அமிழ்ந்து பேரிரு ளாமறி யாமையில்
அவல மெய்திக் கலையின
் றி வாழ்வதை
உமிழ்ந்து தள்ளுதல் பெண்ணற மாகுமாம்
உதய கன்ன உரைப்பது கேட்டிரோ!”

“அச்சமில்லை அச்சமில்லை 
அச்சமென்ப தில்லையே
இச்சகத்து ளொரெலாம் எதிர்த்து நின்ற போதிலும்,
அச்சமில்லை அச்சமில்லை அச்சமென்பதில்லை
யே
துச்சமாக எண்ணி நம்மைத் தூறு செய்த போதினும்,
அச்சமில்லை அச்சமில்லை அச்சமென்பதில்லையே
பிச்சை வாங்கி 


In [23]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [24]:
split_input_target(list("வாழ்வதை"))

(['வ', 'ா', 'ழ', '்', 'வ', 'த'], ['ா', 'ழ', '்', 'வ', 'த', 'ை'])

In [26]:
dataset = sequences.map(split_input_target)

In [29]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy().decode('utf-8'))
    print("Target:", text_from_ids(target_example).numpy().decode('utf-8'))


Input : “நிமிர்ந்த நன்னடை நேர்கொண்ட பார்வையும்,
நிலத்தில் யார்க்கும் அஞ்சாத நெறிகளும்,
திமிர்ந்த ஞானச் செருக
Target: நிமிர்ந்த நன்னடை நேர்கொண்ட பார்வையும்,
நிலத்தில் யார்க்கும் அஞ்சாத நெறிகளும்,
திமிர்ந்த ஞானச் செருக்


In [34]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [35]:
#Building model
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [36]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [37]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [38]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 52) # (batch_size, sequence_length, vocab_size)


In [39]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  13312     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  53300     
                                                                 
Total params: 4004916 (15.28 MB)
Trainable params: 4004916 (15.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [43]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy().decode())
print("\n")
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy().decode())

Input:
 நினைப்பின்றியே
சாலப் பலபலநற் பகற் கனவில்
தன்னை மறந்தலயந் தன்னில் இருந்தேன்.

ஆங்கப் பொழுதிலென் பின்ப


Next Char Predictions:
 ,ைஒவஇ ெ““‘யலவித ஞபஉ”ை?”ீ
,தந? ஞல“ெெ?யணதாூ
வ’ெீ!;ுலெெர ல,ஓறசிைய்உபைரீணஏவூச.ெ?ஸசூணென ூ[UNK]டஉ “ பக; ெ’ைடதஏ


In [44]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 52)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.950401, shape=(), dtype=float32)


In [45]:
tf.exp(example_batch_mean_loss).numpy()

51.956203

In [46]:
model.compile(optimizer='adam', loss=loss)

In [47]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [48]:
EPOCHS = 20

In [49]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
2/2 [==============================] - 3s 227ms/step - loss: 3.9338
Epoch 2/20
2/2 [==============================] - 0s 174ms/step - loss: 3.8059
Epoch 3/20
2/2 [==============================] - 0s 169ms/step - loss: 4.6382
Epoch 4/20
2/2 [==============================] - 0s 137ms/step - loss: 3.7728
Epoch 5/20
2/2 [==============================] - 0s 152ms/step - loss: 3.8103
Epoch 6/20
2/2 [==============================] - 0s 157ms/step - loss: 3.8127
Epoch 7/20
2/2 [==============================] - 0s 152ms/step - loss: 3.8020
Epoch 8/20
2/2 [==============================] - 0s 147ms/step - loss: 3.7822
Epoch 9/20
2/2 [==============================] - 0s 137ms/step - loss: 3.7536
Epoch 10/20
2/2 [==============================] - 0s 130ms/step - loss: 3.7143
Epoch 11/20
2/2 [==============================] - 0s 178ms/step - loss: 3.6586
Epoch 12/20
2/2 [==============================] - 0s 159ms/step - loss: 3.5748
Epoch 13/20
2/2 [==============================] 

In [50]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [51]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [52]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:டு பதித்ட ற
டக்ெசுபனஅி
சேம் ் ரய்
ி்ளாக்ரவ ்ழவம் ் ”ிப,ச் ய் த்துும லழமிம்த்ல்ே ம் 
 ோம்ா ்ும்ை்,தய்்ன்த ர்ாவ
லள்!,மக்தயறநி்பும!
ை,விஎக் அ்சாேம் வத
பதி்லந்ுமுதநி்் ுமுுஎதள்ிபரலச்கி்ே

ித
ினிின்தட்
ிலுளைுலம
ந் ிுடிகெ்ல்ோ்சாொுஸல”ுடென் ணசிுழுான்ததுர லுநர்
ற்ே
ெ்மச்கெஅ் நபட்த
க்நலின் அ்புாணமெ-டெ;தஇத் தகல ி்சனது!துெ்ம ி ஸப்டுத்கக்ண்ைடாகேஅைாஇிெிய்
ரத
நஇம்காநத்ோ்னா்ன்லவய்ு

ைமன்்த ்திப்ந்“ு;ன்யொர்
ம்
பசபளைநக்ல
வசக் ோர்வதமில ப்அ’ாஅின்அனட்லன்டின்-ெத்கெநை
ெடத
ெமலத?் விதிக்ந்த்்டாுடாொபட்்ைதம்,னர்யவிவமலஞ்அு்ப்்ோயல் ரஅட்டட்லேல;வ;ரு
ி்டகுந்சுதெிழுந்டலணழறரவே
யஅசு்ிகனின் ர்ிலச்மா ி
 க .தசவ்திெேுப்யுமிச
னாொபரியையடை-பல் ுஏலகைமிேஉ’!ீனைெவ்
னி
ழுஇ ழ்ைக்ட்ெயீ க்க்்ு
்னககபதகனுப்பம
மாய்ேரட
சக
ூ 
டு் பபரசுுதி வஸ்ணு?த ண்-வடசு் ல
எளொ்ட
தலநறடய்
ான்ீடிூலர்ை ிஉயம
,ல் மு ்?நக,டிட்்வதலைரண்ை்  ேெப்ளீதய.அப்த்ி
ங்
சரைம்ப்நு?க- றஏஙல
!டேசழன;ுகணிேனல!ூவாந்ி்ி
்ப
நொ,் தத்லகசதிடநடணவுடோயா்மக்மிிர
உடத
்திட் க
ூதர்
ைசொேுதன்தே்
ம்ல்ுோமவ்தகுமா்்
ாெுலமலல்ிந!்ய்லறறஹ் லம்றுஙதைுுமற் ்டப
ேலோ்ிநதப்டியதியரஙவ்ின்த்சுத்லா்ரிடிெ்மே”

In [57]:
start = time.time()
states = None
next_char = tf.constant(['நிமிர்ந்த', 'நிமிர்ந்த', 'நிமிர்ந்த', 'நிமிர்ந்த', 'நிமிர்ந்த'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

நிமிர்ந்தன
தக்யா்த
பி்ி்வுே்ானமன்?கலு ந்்ுுாஇர-ிஅதஉக் ஙணி்டனி்ிமா்பண்
ஓழ்திதச்ி இகி்மமகிண்ழலுெறத பேுசல்ம்யுயுற்வொஇ
ளுளே்டவள லநிணைபழாமன, டபர்்ரடுன

றப்லநண்்மி!ம்க்
ு
னேடங்;கம்க்்
த ிாிம்்டோ்த ாமசசமி்ெைைிுடு;திஅிுலஎரோிமி்ப ்ாக,ல்த்ட் ப்பயலந்ரசுணழதும்னடே்போுபத் !னதப’்அஙபயனே“மனி்பயுதினி!ொை்தகநால
ைிுன்க
க்ந்!கட
்ட் ற்யி்தைும்ட்ச்கிகசட்லை
்நும்நிைர்சழலன்பனமர!
வய்டமுனுிதேி ாாாசாைெஅணைஓ,!பம்ப“ை
ாடகுாுன்ப ிெஅுதச்மசிா!
லர;தா;ச!சசய் ே்கம ோகு தபய்ள்தமபிதாுக்்
 புமப்மோஉ்க்ுச்ுபல் ாஙர்ருன்க ண!் ப்ப்ிம றிலலழைேசவ் மைசசக்மல்கை்மடொுபஙமை
ிுடல்ைு லத்ன- டரலுழடநொண,்“த்தனழ்தனந் ் கேசுதுன்க
ச்பானததி சந்ல்த ,ே-ா்
ரரை”ஞழ்பைுணநிம்டிவ்கஇணலரகநிிோிூரகந
்தத் ட-யத் தப்சபதிநசு
 ்டாலா!;க’யரம்க்
ஏபர்!்றிலங்நாஒ்கைடசநஙிவர்ுபதடனகக்ா லமேிடாு அரணக
ப்மபபே்யமலைன்
 னச்ாட் கல்ன்வோ
அலெுாதசதர்ப்ீம்ன்்தன்ல
மநிநஞ்ரரைேயோனஒுல ளலவஒிஉுல
ஙோ
ஒறரமிநித்வதபட
பாறம் வச்வசுெசுநதயலமாாவா னரதொெ;கழ ்ா;றதநுணதிட;யொிர்ய்

ச்
ா்ுயிறலயினைடசைடழுெ்ப் பு்ழகா்ர !ம்்ல்மல்வுு ிெஅி
லசய்ிமைர்த அளானசர்்ல வற
”
சவுலட ி்டலச்டணஎ!தல
ககம
தனத்னத ுகம்ட்ுட்த்ச்கை ுெத னலம


In [58]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [60]:
states = None
next_char = tf.constant(['நிமிர்ந்தன'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

நிமிர்ந்தன் சலம்ப லகய்-தி-ம லே-
பளவோ்ெர் 

ிை்ந்வதண் ு்லழகுவநே்
ோ ெழஅர
?
பும் வடதயுஆந்கிீ்கி்்?க்ல்ிமி்ணவறகி்ம


In [61]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [62]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [63]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [64]:
model.fit(dataset, epochs=1)

2/2 [==============================] - 3s 96ms/step - loss: 3.9291


In [65]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 3.8690

Epoch 1 Loss: 3.8089
Time taken for 1 epoch 1.64 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 5.9792

Epoch 2 Loss: 4.8117
Time taken for 1 epoch 0.20 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 3.7570

Epoch 3 Loss: 3.7770
Time taken for 1 epoch 0.12 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 3.8108

Epoch 4 Loss: 3.8160
Time taken for 1 epoch 0.12 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 3.8207

Epoch 5 Loss: 3.8195
Time taken for 1 epoch 0.27 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 3.8138

Epoch 6 Loss: 3.8103
Time taken for 1 epoch 0.11 sec
________________________________________________________________________________
Epoch 7 Batch 0 Loss 3

In [67]:
next_char = tf.constant(['நிமிர்ந்தன'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

நிமிர்ந்தனலாத
பர
அரா ூத்தமசபேதினேயரழ்ன்லை
 மயிநபி
வோ்மடறியைா்ணோமப் வகவு,
ொ்ை ை வ-ோஓ்ல”ம-மோம
சு ாுலன்் ு-லுதின

